In [10]:
#!pip install py7zr 
import geopandas as gpd
import pandas as pd
import requests
from bs4 import BeautifulSoup
import zipfile
import py7zr
import os
import shutil
import getpass

#### Gestion du proxy

In [16]:
http_proxy = getpass.getpass(prompt='http_proxy')

http_proxy ········


In [17]:
https_proxy = getpass.getpass(prompt='https_proxy')

https_proxy ········


In [18]:
req = "https://geoservices.ign.fr/ocsge"

In [19]:
proxies = { "http"  : http_proxy, "https" : https_proxy}

In [20]:
page = requests.get(req, proxies=proxies)

In [21]:
soup = BeautifulSoup(page.text, 'html.parser')

In [22]:
downloads = soup.findAll('a', href=True)

Liste des départements disponibles

In [23]:
liste_ocsge = [download for download in downloads if "OCS-GE_2-0" in download['href']]
liste_ocsge = pd.DataFrame({ 'name' : [item.find('strong').text if item.find('strong') else None for item in liste_ocsge],
               'link' : [item['href'] for item in liste_ocsge] })


liste_ocsge['codgeo'] = liste_ocsge.name.str.split(" - ",expand=True)[0].str.slice(12,).str.strip()
liste_ocsge['year'] = liste_ocsge.name.str.extract(r'(\b\d{4}\b)')
temp = liste_ocsge.name.str.extract(r'(\b\d{4}-\d{4}\b)')
liste_ocsge.loc[~temp.isna().any(axis=1),'year'] = temp[~temp.isna().any(axis=1)][0]
liste_ocsge = liste_ocsge[['codgeo', 'year', 'link', 'name']]


In [24]:
liste_ocsge.head()

,codgeo,year,link,name
0,01,2021,https://data.geopf.fr/telechargement/download/...,Département 01 - Ain - 2021
1,01,2018,https://data.geopf.fr/telechargement/download/...,Département 01 - Ain - 2018
2,01,2018-2021,https://data.geopf.fr/telechargement/download/...,Département 01 - Ain - différentiel - 2018-2021
3,38,2021,https://data.geopf.fr/telechargement/download/...,Département 38 - Isère - 2021
4,38,2018,https://data.geopf.fr/telechargement/download/...,Département 38 - Isère - 2018


Traitement sur des départements spécifiques

In [26]:
#liste_dep = ['09', '11', '12', '30', '31', '32', '34', '46', '48', '65', '66', '81', '82']

In [30]:
#liste_ocsge_dep = liste_ocsge[liste_ocsge.codgeo.isin(liste_dep)].copy()
#liste_ocsge_dep = liste_ocsge_dep[liste_ocsge_dep.year.str.len()==4]
liste_ocsge_dep = liste_ocsge.copy()

Traitement des données annuels

In [31]:
liste_ocsge_dep = liste_ocsge_dep[liste_ocsge_dep.year.str.len()==4]
liste_ocsge_dep = liste_ocsge_dep[:20]
liste_ocsge_dep.head()

,codgeo,year,link,name
0,01,2021,https://data.geopf.fr/telechargement/download/...,Département 01 - Ain - 2021
1,01,2018,https://data.geopf.fr/telechargement/download/...,Département 01 - Ain - 2018
3,38,2021,https://data.geopf.fr/telechargement/download/...,Département 38 - Isère - 2021
4,38,2018,https://data.geopf.fr/telechargement/download/...,Département 38 - Isère - 2018
6,42,2022,https://data.geopf.fr/telechargement/download/...,Département 42 - Loire - 2022


In [ ]:
for url in liste_ocsge_dep.link:
  with requests.get(url, stream=True, proxies=proxies) as response:
    if response.status_code == 200:
        with open(url.rsplit('/', 1)[1], 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

  search_string = "OCCUPATION_SOL"
  with py7zr.SevenZipFile(url.rsplit('/', 1)[1], mode='r') as archive:
        # Get the list of files in the archive
        all_files = archive.getnames()

        # Filter files that contain the search string
        filtered_files = [f for f in all_files if search_string in f]

        # Extract only the filtered files
        if filtered_files:
          archive.extract(targets=filtered_files, path='ocsge_temp',recursive=False)

        for file in filtered_files:
                source_file = os.path.join('ocsge_temp', file)
                destination_file = os.path.join('ocsge', os.path.basename(file))
                os.makedirs(os.path.dirname(destination_file), exist_ok=True)
                if os.path.exists(source_file):  # Only move if the file was extracted
                    os.rename(source_file, destination_file)

  ocsge = gpd.read_file('ocsge/OCCUPATION_SOL.shp')
  ocsge.to_parquet('files_ocsge/' + url.rsplit('/', 1)[1].rsplit('.', 1)[0]+".parquet")

  shutil.rmtree('ocsge_temp')
  shutil.rmtree('ocsge')
  os.remove(url.rsplit('/', 1)[1])